## 1. 環境確認とライブラリのインポート

In [ ]:
import os
from pathlib import Path
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import csv

# PyTorchバージョンとデバイス確認
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

if device.type == "mps":
    print("\n⚠️ MPS support is preliminary. SAM 2 might give numerically different outputs.")

## 2. 設定パラメータ

ここで解析したいビデオや出力先を指定します。

In [ ]:
# 作業ディレクトリ
ROOT = Path.cwd()

# 入力ビデオ設定
VIDEO_MP4 = ROOT / "videos" / "D1AT_20251029_Trim_night_30sec.mp4"
FRAMES_DIR = ROOT / "videos" / "night_30sec"

# SAM2 モデル設定
CONFIG_NAME = "configs/sam2.1/sam2.1_hiera_l.yaml"  # Hydraパス
CHECKPOINT = ROOT / "sam2" / "checkpoints" / "sam2.1_hiera_large.pt"

# 出力設定
OUTPUT_CSV = ROOT / "positions" / "mouse_tracks_night.csv"
SEGMENTED_DIR = FRAMES_DIR.parent / f"{FRAMES_DIR.name}_segmented"

# トラッキング設定
NUM_OBJECTS = 2  # トラッキングするマウスの個体数
ANN_FRAME_IDX = 0  # クリック指定するフレーム番号

print(f"Video: {VIDEO_MP4}")
print(f"Frames dir: {FRAMES_DIR}")
print(f"Checkpoint: {CHECKPOINT}")
print(f"Output CSV: {OUTPUT_CSV}")

## 3. フレーム抽出（必要な場合のみ）

ビデオからJPEGフレームを抽出します。既にフレームがある場合はスキップされます。

In [ ]:
def ensure_frames(frames_dir: Path, video_path: Path) -> None:
    """ビデオからフレーム画像を抽出（既にある場合はスキップ）"""
    if frames_dir.exists():
        if any(frames_dir.glob("*.jpg")) or any(frames_dir.glob("*.jpeg")):
            print(f"Frames already exist in {frames_dir}")
            return
    
    if not video_path.exists():
        raise FileNotFoundError(f"Video not found: {video_path}")
    
    frames_dir.mkdir(parents=True, exist_ok=True)
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {video_path}")
    
    index = 0
    print(f"Extracting frames from {video_path} to {frames_dir} ...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = frames_dir / f"{index:05d}.jpg"
        cv2.imwrite(str(frame_path), frame)
        index += 1
    cap.release()
    
    if index == 0:
        raise RuntimeError(f"No frames extracted from {video_path}")
    print(f"✅ Extracted {index} frames.")

# フレーム抽出実行
ensure_frames(FRAMES_DIR, VIDEO_MP4)

## 4. フレーム一覧の取得

In [ ]:
def _frame_sort_key(path: Path):
    stem = path.stem
    return int(stem) if stem.isdigit() else stem

frame_paths = sorted(
    [p for p in FRAMES_DIR.iterdir() if p.suffix.lower() in {".jpg", ".jpeg"}],
    key=_frame_sort_key,
)

if not frame_paths:
    raise RuntimeError(f"No JPEG frames found in {FRAMES_DIR}")

print(f"✅ Found {len(frame_paths)} frames")

## 5. SAM2モデルの読み込み

In [ ]:
from sam2.build_sam import build_sam2_video_predictor

if not CHECKPOINT.exists():
    raise FileNotFoundError(f"Checkpoint not found: {CHECKPOINT}")

predictor = build_sam2_video_predictor(CONFIG_NAME, str(CHECKPOINT), device=device)
print("✅ SAM2 model loaded")

## 6. ヘルパー関数の定義

In [ ]:
def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=200):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', 
               s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', 
               s=marker_size, edgecolor='white', linewidth=1.25)

## 7. 推論状態の初期化

In [ ]:
inference_state = predictor.init_state(video_path=str(FRAMES_DIR))
print("✅ Inference state initialized")

## 8. 対象オブジェクトのクリック指定

**重要:** OpenCVウィンドウが開くので、各マウスをクリックしてからEnterキーを押してください。

In [ ]:
class Pointer:
    def __init__(self, img_path: Path):
        self.img_path = img_path
        self.img = cv2.imread(str(img_path))
        if self.img is None:
            raise FileNotFoundError(f"Failed to load image: {img_path}")
        self.x = 0
        self.y = 0
        self._clicked = False

    def onMouse(self, event, x, y, flags, params):
        if event == cv2.EVENT_LBUTTONDOWN:
            print(f"Clicked at ({x}, {y})")
            self.x = x
            self.y = y
            self._clicked = True
            self.img_marked = self.img.copy()
            cv2.circle(self.img_marked, (self.x, self.y), 8, (0, 0, 255), -1)
            cv2.imshow(str(self.img_path), self.img_marked)

    def point_gui(self):
        cv2.imshow(str(self.img_path), self.img)
        cv2.setMouseCallback(str(self.img_path), self.onMouse)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        if not self._clicked:
            raise RuntimeError("No click detected. Please click before closing.")

prompts = {}

for obj_idx in range(NUM_OBJECTS):
    ann_obj_id = obj_idx + 1
    print(f"\n📍 Please click on object {ann_obj_id}/{NUM_OBJECTS} in the window, then press Enter")
    
    pointer = Pointer(frame_paths[ANN_FRAME_IDX])
    pointer.point_gui()
    
    points = np.array([[pointer.x, pointer.y]], dtype=np.float32)
    labels = np.array([1], np.int32)
    prompts[ann_obj_id] = points, labels
    
    _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=ANN_FRAME_IDX,
        obj_id=ann_obj_id,
        points=points,
        labels=labels,
    )
    
    # 確認用の可視化
    plt.figure(figsize=(9, 6))
    plt.title(f"Object {ann_obj_id} - Frame {ANN_FRAME_IDX}")
    plt.imshow(Image.open(frame_paths[ANN_FRAME_IDX]))
    for i, out_obj_id in enumerate(out_obj_ids):
        show_points(*prompts[out_obj_id], plt.gca())
        show_mask((out_mask_logits[i] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_id)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

print(f"\n✅ Clicked {NUM_OBJECTS} objects")

## 9. 全フレームへの伝播とセグメンテーション

全フレームにわたりセグメンテーションを実行し、結果画像を保存します。

In [ ]:
# 保存ディレクトリ作成
SEGMENTED_DIR.mkdir(parents=True, exist_ok=True)

# 全フレームへ伝播
video_segments = {}
print("\n🔄 Propagating masks through video...")
for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
    video_segments[out_frame_idx] = {
        out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

print(f"✅ Propagation complete for {len(video_segments)} frames")

# セグメンテーション画像の出力
print("\n💾 Saving segmented images...")
plt.close("all")
for out_frame_idx in tqdm(range(len(frame_paths)), desc="Saving images"):
    plt.figure(figsize=(6, 4))
    plt.title(f"Frame {out_frame_idx}")
    plt.imshow(Image.open(frame_paths[out_frame_idx]))
    
    if out_frame_idx in video_segments:
        for out_obj_id, out_mask in video_segments[out_frame_idx].items():
            show_mask(out_mask, plt.gca(), obj_id=out_obj_id)
    
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(SEGMENTED_DIR / f"s_{frame_paths[out_frame_idx].name}", bbox_inches='tight')
    plt.close()

print(f"✅ Saved segmented images to {SEGMENTED_DIR}")

## 10. 重心座標の計算と保存

In [ ]:
tracks = defaultdict(list)

for frame_idx, obj_dict in video_segments.items():
    for obj_id, mask in obj_dict.items():
        m = mask.squeeze()
        if m.sum() == 0:
            continue
        ys, xs = np.nonzero(m)
        cx = xs.mean()
        cy = ys.mean()
        tracks[obj_id].append((frame_idx, float(cx), float(cy)))

# CSV保存
OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
with OUTPUT_CSV.open("w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["object_id", "frame_idx", "cx", "cy"])
    for obj_id, traj in tracks.items():
        for frame_idx, cx, cy in traj:
            writer.writerow([obj_id, frame_idx, cx, cy])

print(f"✅ Saved centroid CSV to {OUTPUT_CSV}")

## 11. 軌跡の可視化

保存したCSVから軌跡を読み込んで可視化します。

In [ ]:
# CSVの読み込み
df = pd.read_csv(OUTPUT_CSV)

# フレーム番号を時間に変換
FPS = 25.0
df["time"] = df["frame_idx"] / FPS

print(f"Loaded {len(df)} data points for {df['object_id'].nunique()} objects")
df.head()

### 11.1 時系列プロット（x座標とy座標）

In [ ]:
palette = ["orange", "green", "blue", "purple", "red", "brown", "magenta", "cyan"]

fig, axes = plt.subplots(2, 1, sharex=True, figsize=(12, 6))

for idx, (obj_id, g) in enumerate(df.groupby("object_id")):
    color = palette[idx % len(palette)]
    axes[0].plot(g["time"], g["cx"], label=f"Mouse {obj_id}", color=color)
    axes[1].plot(g["time"], g["cy"], label=f"Mouse {obj_id}", color=color)

axes[0].set_ylabel("x (pixels)")
axes[1].set_ylabel("y (pixels)")
axes[1].set_xlabel("Time (s)")
axes[0].set_title("X coordinate vs Time")
axes[1].set_title("Y coordinate vs Time")
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 11.2 2D軌跡プロット

In [ ]:
plt.figure(figsize=(8, 6))

for idx, (obj_id, g) in enumerate(df.groupby("object_id")):
    color = palette[idx % len(palette)]
    plt.plot(g["cx"], g["cy"], marker=".", linestyle="-", 
             label=f"Mouse {obj_id}", color=color, alpha=0.7)

plt.xlabel("x (pixels)")
plt.ylabel("y (pixels)")
plt.title("2D Trajectories")
plt.gca().invert_yaxis()  # 画像座標系に合わせる
plt.grid(True, alpha=0.3)
plt.legend()
plt.axis('equal')
plt.tight_layout()
plt.show()

## 完了

すべての処理が完了しました！

### 出力ファイル
- セグメンテーション画像: `videos/night_30sec_segmented/`
- 重心座標CSV: `positions/mouse_tracks_night.csv`

### 次のステップ
- CSVファイルをExcelや他のツールで解析
- より詳細な統計解析（移動距離、速度など）
- 異なるビデオでの解析